In [1]:
# 查看GPU信息
!nvidia-smi

Fri May 13 06:38:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:06:00.0 Off |                    0 |
| N/A   33C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="DXGaQFBdatLlmobAlwRW")
project = rf.workspace("graduation-project-workspace").project("data-zlgg9")
dataset = project.version(4).download("coco")

Looking in indexes: https://mirrors.sjtug.sjtu.edu.cn/pypi/web/simple
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to data-4 in coco:: 100%|██████████| 3063/3063 [00:01<00:00, 2434.53it/s]


In [11]:
# 使用过程发现Roboflow部分图片json信息中的尺寸不正确
# 在这里修改
import json
from PIL import Image

prefixes = ["data-4/train/", "data-4/test/", "data-4/valid/"]
for prefix in prefixes:
    with open(prefix + "_annotations.coco.json", "r") as f:
        data = json.load(f)
    
    images = data["images"]
    for image in images:
        file_path = image["file_name"]
        img_file = Image.open(prefix + file_path)
        image["height"] = img_file.height
        image["width"] = img_file.width

    %rm -rf {prefix}_annotations.coco.json
    with open(prefix + "_annotations.coco.json", "w") as f:
        json.dump(data, f)

In [6]:
# 执行后重启运行时，让安装的依赖生效
# 安装Detectron2
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
Looking in indexes: https://mirrors.sjtug.sjtu.edu.cn/pypi/web/simple
Obtaining file:///faster/detectron2_repo
  Running setup.py develop for detectron2


In [1]:
import os
print(os.getcwd())
%cd faster

/
/faster


In [2]:
# 一些基础设置
# 配置 detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import依赖库
import numpy as np
import cv2
import random

# import detectron2工具
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

/root/miniconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 向detectron2注册数据集
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/faster/data-4/train/_annotations.coco.json", "/faster/data-4/train")
register_coco_instances("my_dataset_val", {}, "/faster/data-4/valid/_annotations.coco.json", "/faster/data-4/valid")
register_coco_instances("my_dataset_test", {}, "/faster/data-4/test/_annotations.coco.json", "/faster/data-4/test")

In [4]:
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

WARNING [05/13 07:36:40 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[05/13 07:36:40 d2.data.datasets.coco]: Loaded 2670 images in COCO format from /faster/data-4/train/_annotations.coco.json


In [5]:
# 在开始培训之前，我们需要确保模型根据我们的验证集进行验证
# 需要根据DefaultTrainer和COCOEvaluator一起定义实现
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [9]:
%load_ext tensorboard
%tensorboard --logdir output

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
# Faster RCNN 的 modelzoo https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines
from detectron2.config import get_cfg
import os

cfg = get_cfg()
# 下面模型显存不足，改用较小的
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 4
# 加载预训练权重
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
# 一个Batch的图片数量
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001


# 预热训练循环
cfg.SOLVER.WARMUP_ITERS = 1000
# SGD迭代的最多次数
# 如果mAP仍然上升调高训练循环，过拟合则调低
cfg.SOLVER.MAX_ITER = 20000
# 'steps_with_decay' 和 'steps_with_lrs'策略非均匀化步长 Non-uniform step iterations
cfg.SOLVER.STEPS = (2000, 5000, 10000)
# 学习率衰减因子，对于 'step',在每一步，当前 LR 乘以因子 SOLVER.GAMMA
cfg.SOLVER.GAMMA = 0.05

# 每张图片选择的element数量
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
# 类别数 + 1（加上背景）
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

# 验证周期
cfg.TEST.EVAL_PERIOD = 500


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[05/13 08:05:58 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (6, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (6,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (20, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (20,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[05/13 08:05:59 d2.engine.train_loop]: Starting training from iteration 0
[05/13 08:06:50 d2.utils.events]:  eta: 13:50:54  iter: 19  total_loss: 2.54  loss_cls: 1.886  loss_box_reg: 0.5556  loss_rpn_cls: 0.037  loss_rpn_loc: 0.01718  time: 2.5503  data_time: 0.0547  lr: 1.9981e-05  max_mem: 8599M
[05/13 08:07:41 d2.utils.events]:  eta: 14:25:00  iter: 39  total_loss: 2.033  loss_cls: 1.481  loss_box_reg: 0.4389  loss_rpn_cls: 0.03299  loss_rpn_loc: 0.01244  time: 2.5473  data_time: 0.0240  lr: 3.9961e-05  max_mem: 8599M
[05/13 08:08:34 d2.utils.events]:  eta: 14:33:30  iter: 59  total_loss: 1.686  loss_cls: 0.9989  loss_box_reg: 0.502  loss_rpn_cls: 0.1159  loss_rpn_loc: 0.03393  time: 2.5736  data_time: 0.0195  lr: 5.9941e-05  max_mem: 8599M
[05/13 08:09:26 d2.utils.events]:  eta: 14:26:12  iter: 79  total_loss: 1.519  loss_cls: 0.7215  loss_box_reg: 0.6528  loss_rpn_cls: 0.04963  loss_rpn_loc: 0.02326  time: 2.5773  data_time: 0.0209  lr: 7.9921e-05  max_mem: 8599M
[05/13 08:10:16 d